# Custom_train_test_split

In [ ]:
def custom_train_test_split(X, y, test_size, random_state):
    df = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis = 1)
    test_len = rount(len(df)*test_size)
    if random_state <= len(df) - test_len:
        test_df = df[random_state : random_staet + test_len]
        train_df = df[:random_state] + df[random_state + test_len:]
    else:
        random_state = 0
        df.sample(frac = 1)
    return train_df[:-1], test_df[:-1], train_df[-1], test_df[-1]

# Custom_KFold

In [ ]:
def custom_KFold(model, data, cv):
    acc_lst = []
    data = pd.DataFrame(data.sample(frac = 1))
    initial_ratio = round(1 / cv)*len(data)
    ratio = initial_ratio
    
    for i in range(cv):
        val_data = data[i*initial_ratio : ratio]
        train_data = data.drop(data[i*initial_ratio:ratio], axis = 0)
        
        X_train, X_test, y_train, y_test = train_data[:-1], train_data[-1], val_data[:-1], val_data[-1]
        
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        acc_lst.append(accuracy_score(y_test, pred))
    return acc_lst

# Dacon_Sports_Example

In [ ]:
df_train = pd.read_csv("drive/MyDrive/dataset/Dacon/fifa/FIFA_train.csv", index_col = 0)
df_test = pd.read_csv("drive/MyDrive/dataset/Dacon/fifa/FIFA_test.csv", index_col = 0)

df_train["contract_left"] = 0
df_train["contract_until"] = pd.to_numeric(df_train["contract_until"])
df_train["contract_left"] = df_train["contract_until"] - df_train["contract_until"].min()

scaler = StandardScaler()
df_train["stat_overall"] = scaler.fit_transform(np.array(df_train.stat_overall).reshape(-1, 1))
df_train["stat_potential"] = scaler.fit_transform(np.array(df_train.stat_potential).reshape(-1, 1))
df_train["stat_skill_moves"] = scaler.fit_transform(np.array(df_train.stat_skill_moves).reshape(-1, 1))

df_train = pd.get_dummies(df_train, columns = ["continent", "prefer_foot", "position"])
df_test = pd.get_dummies(df_test, columns = ["continent", "prefer_foot", "position"])

rf_reg = RandomForestRegressor()
lgbm_reg = LGBMRegressor()
xgbm_reg = XGBRegressor()
cat_reg = CatBoostRegressor()
ng_reg = NGBRegressor()

models = [rf_reg, lgbm_reg, xgbm_reg, cat_reg, ng_reg]
rmse = []
pred_df = pd.DataFrame()

kfold = KFold(n_splits= 10, shuffle = True, random_state = 23)

test_result = np.zeros((target.shape[0]))

for model in models:
    
    temp = []

    for train_idx, test_idx in kfold.split(X, y):
                
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

        model.fit(X_train, np.log1p(y_train))
        pred = model.predict(X_test)
        score = np.sqrt(mean_squared_error(np.expm1(pred), y_test))

        test_pred = np.expm1([0 if x<0 else x for x in model.predict(target)]) / 10
        test_result += test_pred / 5
    
        temp.append(score) 

    rmse.append(np.mean(temp))
    
    